In [1]:
import os
import random
import shutil
import time
import warnings

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim

import torch.utils.data
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models

In [2]:
SEED=1


In [3]:
random.seed(SEED)
torch.manual_seed(SEED)
cudnn.deterministic = True

In [4]:
START_EPOCH = 0

### Set the architecture to resnet 18 below

In [35]:
##########################
ARCH = 'resnet18'# set the architecture to RESNET 18
# please look up how to do that
########################
EPOCHS = 5
LR = 0.1
MOMENTUM = 0.9
WEIGHT_DECAY = 5e-4
PRINT_FREQ = 50
TRAIN_BATCH=128
VAL_BATCH=128
WORKERS=2
# TRAINDIR="/workspace/data/imagenet2012/train"
# VALDIR="/workspace/data/imagenet2012/val"

In [6]:
# TRAINDIR="/CINIC/train"
# VALDIR="/CINIC/valid"

### Check if cuda is available here

In [7]:
# check if cuda is available in this cell
# if it is not available, you should not go forward!
torch.cuda.is_available()

True

### Assign your GPU below

In [8]:
# Assign your GPU in this cell
GPU = 0

In [9]:
# set your active device to your GPU in this cell
torch.cuda.current_device()

0

In [10]:
# enable algorithm optimization
cudnn.benchmark = True

### Fill in the heart of the train section below

In [11]:
def train(train_loader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter('Time', ':6.3f')
    data_time = AverageMeter('Data', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(train_loader),
        [batch_time, data_time, losses, top1, top5],
        prefix="Epoch: [{}]".format(epoch))

    ######################
    # switch model to train mode here
    model.train()
    ################

    end = time.time()
    for i, (images, target) in enumerate(train_loader):
        # measure data loading time
        data_time.update(time.time() - end)

        #####################
        # send the images to cuda device
        # send the target to cuda device
        images = torch.tensor(images).to('cuda')
        target = torch.tensor(target).to('cuda')


        # compute output
        # output = model ?? images
        output = model(images)

        # compute loss 
        # loss = criterion, output, target
        loss = criterion(output, target)

        # measure accuracy and record loss
        acc1, acc5 = accuracy(output, target, topk=(1, 5))
        losses.update(loss.item(), images.size(0))
        top1.update(acc1[0], images.size(0))
        top5.update(acc5[0], images.size(0))

        # compute gradient and do SGD step
        
        #### zero out gradients in the optimier
        ## optimizer ..??
        optimizer.zero_grad()

        ## backprop!
        ### loss... ???
        loss.backward()

        # update the weights!
        ## optimier .. ??
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % PRINT_FREQ == 0:
            progress.display(i)

#### Fill in the validate section below

In [12]:
def validate(val_loader, model, criterion):
    batch_time = AverageMeter('Time', ':6.3f')
    losses = AverageMeter('Loss', ':.4e')
    top1 = AverageMeter('Acc@1', ':6.2f')
    top5 = AverageMeter('Acc@5', ':6.2f')
    progress = ProgressMeter(
        len(val_loader),
        [batch_time, losses, top1, top5],
        prefix='Test: ')

    # switch to evaluate mode
    # model ???
    model.eval()

    with torch.no_grad():
        end = time.time()
        for i, (images, target) in enumerate(val_loader):
            ### send the images and target to cuda
            images = torch.tensor(images).to('cuda')
            target = torch.tensor(target).to('cuda')

            # compute output
            # output = model ??? images?
            output = model(images)

            # compute loss
            # loss  = criterion ?? output ?? target
            loss = criterion(output, target)

            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            losses.update(loss.item(), images.size(0))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % PRINT_FREQ == 0:
                progress.display(i)

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

    return top1.avg

### Save the checkpoint

In [13]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    # save the model state!
    # state ???
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

In [14]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)

In [15]:
class ProgressMeter(object):
    def __init__(self, num_batches, meters, prefix=""):
        self.batch_fmtstr = self._get_batch_fmtstr(num_batches)
        self.meters = meters
        self.prefix = prefix

    def display(self, batch):
        entries = [self.prefix + self.batch_fmtstr.format(batch)]
        entries += [str(meter) for meter in self.meters]
        print('\t'.join(entries))

    def _get_batch_fmtstr(self, num_batches):
        num_digits = len(str(num_batches // 1))
        fmt = '{:' + str(num_digits) + 'd}'
        return '[' + fmt + '/' + fmt.format(num_batches) + ']'

In [16]:
# if we are adjusting the LR manually use this
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = LR * (0.1 ** (epoch // 30))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

In [17]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

In [18]:
imagenet_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
imagenet_std_RGB = [0.229, 0.224, 0.225]
cinic_mean_RGB = [0.47889522, 0.47227842, 0.43047404]
cinic_std_RGB = [0.24205776, 0.23828046, 0.25874835]
cifar_mean_RGB = [0.4914, 0.4822, 0.4465]
cifar_std_RGB = [0.2023, 0.1994, 0.2010]

In [19]:
normalize = transforms.Normalize(mean=cifar_mean_RGB, std=cifar_std_RGB)

In [20]:
IMG_SIZE = 32
# IMG_SIZE = 224

### Initialize the model using the architecture you selected above

In [21]:
# select the model
# model = ...
model = models.get_model(ARCH)

In [22]:
model.fc = nn.Linear(512, 10)
print(model)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

### Send the model to the cuda device

In [23]:
# send the model to the cuda device..
model = model.to('cuda')

### Instantiate the loss to cross entropy

In [24]:
# use the cross-entropy loss
criterion = nn.CrossEntropyLoss()

### Instantiate the optimizer to SGD

In [25]:
# use SGD .. use the momentum and weight decay vars
optimizer = torch.optim.SGD(model.parameters(), lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)

#### Create the learning rate scheduler

In [26]:
# use CosineAnnealingLR
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=EPOCHS)

In [27]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
])

### Create the train dataset object

In [28]:
# use torchvision.datasets.CIFAR10
train_dataset = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder('data/cinic/train',
    	transform=transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=cinic_mean_RGB,std=cinic_std_RGB)])),
    batch_size=TRAIN_BATCH, shuffle=True)

In [29]:
transform_val = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar_mean_RGB, cifar_std_RGB),
])

### Create the val dataset object

In [30]:
# use torchvision.datasets.CIFAR10
val_dataset = torch.utils.data.DataLoader(
    torchvision.datasets.ImageFolder('data/cinic/valid',
    	transform=transforms.Compose([transforms.ToTensor(),
        transforms.Normalize(mean=cinic_mean_RGB,std=cinic_std_RGB)])),
    batch_size=VAL_BATCH, shuffle=True)

### Create the train dataloader

In [44]:
# fill this in
train_loader = torch.utils.data.DataLoader(train_dataset.dataset, batch_size=TRAIN_BATCH, shuffle=True)

### Create the val dataloader

In [43]:
# fill this in..
val_loader = torch.utils.data.DataLoader(val_dataset.dataset, batch_size=VAL_BATCH, shuffle=False)

In [33]:
best_acc1 = 0

In [45]:
for epoch in range(START_EPOCH, EPOCHS):
#    adjust_learning_rate(optimizer, epoch)

    # train for one epoch
    train(train_loader, model, criterion, optimizer, epoch)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion)

    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)


    save_checkpoint({
        'epoch': epoch + 1,
        'arch': ARCH,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    scheduler.step()
    print('lr: ' + str(scheduler.get_last_lr()))

C:\Users\grandid\AppData\Local\Temp\1\ipykernel_18552\2208984069.py:25: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(images).to('cuda')
C:\Users\grandid\AppData\Local\Temp\1\ipykernel_18552\2208984069.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).to('cuda')


Epoch: [0][  0/704]	Time  4.865 ( 4.865)	Data  0.134 ( 0.134)	Loss 2.6469e+00 (2.6469e+00)	Acc@1   8.59 (  8.59)	Acc@5  38.28 ( 38.28)
Epoch: [0][ 50/704]	Time  0.100 ( 0.193)	Data  0.083 ( 0.085)	Loss 2.2938e+00 (3.4218e+00)	Acc@1  13.28 ( 18.90)	Acc@5  67.97 ( 66.73)
Epoch: [0][100/704]	Time  0.103 ( 0.148)	Data  0.085 ( 0.085)	Loss 2.0075e+00 (2.9091e+00)	Acc@1  26.56 ( 20.86)	Acc@5  85.16 ( 71.48)
Epoch: [0][150/704]	Time  0.099 ( 0.133)	Data  0.083 ( 0.085)	Loss 1.9249e+00 (2.6103e+00)	Acc@1  24.22 ( 23.34)	Acc@5  83.59 ( 75.41)
Epoch: [0][200/704]	Time  0.102 ( 0.125)	Data  0.084 ( 0.084)	Loss 1.7742e+00 (2.4397e+00)	Acc@1  33.59 ( 24.90)	Acc@5  86.72 ( 77.48)
Epoch: [0][250/704]	Time  0.102 ( 0.120)	Data  0.084 ( 0.084)	Loss 1.8089e+00 (2.3185e+00)	Acc@1  32.03 ( 26.23)	Acc@5  84.38 ( 79.21)
Epoch: [0][300/704]	Time  0.102 ( 0.117)	Data  0.084 ( 0.084)	Loss 1.6336e+00 (2.2292e+00)	Acc@1  43.75 ( 27.31)	Acc@5  91.41 ( 80.45)
Epoch: [0][350/704]	Time  0.101 ( 0.115)	Data  0.083 ( 

C:\Users\grandid\AppData\Local\Temp\1\ipykernel_18552\3161145004.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  images = torch.tensor(images).to('cuda')
C:\Users\grandid\AppData\Local\Temp\1\ipykernel_18552\3161145004.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target = torch.tensor(target).to('cuda')


Test: [  0/704]	Time  0.103 ( 0.103)	Loss 1.3248e+00 (1.3248e+00)	Acc@1  60.16 ( 60.16)	Acc@5  84.38 ( 84.38)
Test: [ 50/704]	Time  0.111 ( 0.097)	Loss 8.0303e-01 (1.1987e+00)	Acc@1  75.78 ( 62.68)	Acc@5  95.31 ( 87.06)
Test: [100/704]	Time  0.110 ( 0.103)	Loss 1.4020e+00 (1.2997e+00)	Acc@1  50.00 ( 56.88)	Acc@5  93.75 ( 88.51)
Test: [150/704]	Time  0.110 ( 0.105)	Loss 2.2977e+00 (1.4760e+00)	Acc@1  13.28 ( 49.37)	Acc@5  77.34 ( 87.29)
Test: [200/704]	Time  0.109 ( 0.106)	Loss 2.3501e+00 (1.7013e+00)	Acc@1   4.69 ( 39.66)	Acc@5  76.56 ( 84.17)
Test: [250/704]	Time  0.110 ( 0.107)	Loss 1.5723e+00 (1.7130e+00)	Acc@1  21.88 ( 35.26)	Acc@5  96.09 ( 85.61)
Test: [300/704]	Time  0.109 ( 0.107)	Loss 2.0671e+00 (1.7227e+00)	Acc@1  16.41 ( 32.85)	Acc@5  93.75 ( 87.20)
Test: [350/704]	Time  0.109 ( 0.107)	Loss 2.1988e+00 (1.7623e+00)	Acc@1  13.28 ( 31.20)	Acc@5  91.41 ( 88.11)
Test: [400/704]	Time  0.110 ( 0.108)	Loss 1.3109e+00 (1.7210e+00)	Acc@1  60.94 ( 33.92)	Acc@5  97.66 ( 89.03)
Test: [450